# Runtime Benchmark

In [1]:
import copy
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.transforms import Bbox
from utils.plotting_defaults import (
    autolabel_bars,
    delete_tex_cache,
    parse_logs_long,
    parse_logs_wide,
    plot_legend,
    plot_ylabel,
    set_style,
)

set_style()
Path("plots/runtime_benchmark").mkdir(parents=True, exist_ok=True)

In [2]:
delete_tex_cache()

## Data Loading

In [3]:
execution_list = []
for logfile in Path("../logs/runtime_benchmark/execution/").iterdir():
    config = logfile.stem.split("-")
    data = parse_logs_wide(logfile)

    data["dataset"] = config[0]
    data["query_set"] = config[1]
    data["approach"] = config[2]
    data["execution"] = config[3]

    execution_list.append(data)

execution = pd.DataFrame(
    execution_list,
    columns=[
        "dataset",
        "query_set",
        "approach",
        "execution",
        "query_collection_time",
    ],
)
execution = execution[
    (execution["execution"] == "single") | (execution["execution"] == "single_suppressed")
]

In [4]:
low_selectivity_list = []
for logfile in Path("../logs/runtime_benchmark/low_selectivity/").iterdir():
    config = logfile.stem.split("-")
    data = parse_logs_wide(logfile)

    data["dataset"] = config[0]
    data["query_set"] = config[1]
    data["approach"] = config[2]
    data["execution"] = config[3]

    low_selectivity_list.append(data)

low_selectivity = pd.DataFrame(
    low_selectivity_list,
    columns=[
        "dataset",
        "query_set",
        "approach",
        "execution",
        "query_collection_time",
    ],
)
low_selectivity = low_selectivity[
    (low_selectivity["execution"] == "single")
    | (low_selectivity["execution"] == "single_suppressed")
]

In [5]:
index_trace_list = []
for logfile in Path("../logs/runtime_benchmark/index_trace/").iterdir():
    config = logfile.stem.split("-")
    data = parse_logs_long(logfile)

    for entry in data:
        entry["dataset"] = config[0]
        entry["index_type"] = config[1]

    index_trace_list += data

index_trace = pd.DataFrame(
    index_trace_list,
    columns=[
        "dataset",
        "index_type",
        "metric",
        "value",
    ],
)

In [6]:
indexing_list = []
for logfile in Path("../logs/runtime_benchmark/indexing/").iterdir():
    config = logfile.stem.split("-")
    if len(config) != 4:
        continue
    data = parse_logs_wide(logfile)

    data["dataset"] = config[0]
    data["phase"] = config[1]
    data["parameter"] = config[2][0]
    data["parameter_value"] = int(config[2][1:])

    indexing_list.append(data)

indexing = pd.DataFrame(
    indexing_list,
    columns=[
        "dataset",
        "phase",
        "parameter",
        "parameter_value",
        "total_time",
    ],
)

## Analysis

### Approach Comparison

In [7]:
execution.groupby(["query_set", "dataset", "approach", "execution"]).agg(
    {"query_collection_time": ["mean", "std"]}
)

query_collection_time  \
                                                                       mean   
query_set  dataset       approach   execution                                 
collection gittables     binsort    single                      7905.692510   
                         conversion single                       366.622109   
                                    single_suppressed              0.286408   
                         iterative  single                     48309.523395   
                         rebinning  single                       283.915788   
                                    single_suppressed              0.285870   
           open_data_usa binsort    single                       108.844900   
                         conversion single                         3.911224   
                                    single_suppressed              0.673464   
                         iterative  single                       654.482206   
                         rebinning  single                         5.441237   
                                    single_suppressed              0.663728   
           sportstables  binsort    single                        36.343399   
                         conversion single                         1.583287   
                                    single_suppressed              0.403930   
                         iterative  single                       187.799319   
                         rebinning  single                         1.532570   
                                    single_suppressed              0.404682   
query      gittables     binsort    single                        32.262721   
                         conversion single                         0.377206   
                                    single_suppressed              0.000901   
                         iterative  single                        46.356888   
                         rebinning  single                         0.363129   
                                    single_suppressed              0.000887   
           open_data_usa binsort    single                         0.703077   
                         conversion single                         0.004593   
                                    single_suppressed              0.001128   
                         iterative  single                         1.050508   
                         rebinning  single                         0.005177   
                                    single_suppressed              0.001106   
           sportstables  binsort    single                         0.413263   
                         conversion single                         0.001491   
                                    single_suppressed              0.000580   
                         iterative  single                         0.312364   
                         rebinning  single                         0.001494   
                                    single_suppressed              0.000565   

                                                                    
                                                               std  
query_set  dataset       approach   execution                       
collection gittables     binsort    single               26.340778  
                         conversion single                1.667505  
                                    single_suppressed     0.003362  
                         iterative  single             1074.502817  
                         rebinning  single                2.610721  
                                    single_suppressed     0.001682  
           open_data_usa binsort    single                5.007806  
                         conversion single                0.570307  
                                    single_suppressed     0.010704  
                         iterative  single               26.154671  
                         rebinning  single                0.325634  
      

In [8]:
height = 0.8
for dataset in ["sportstables", "open_data_usa", "gittables"]:
    fig, ax = plt.subplots(1, 1, figsize=(1.2, height))
    data = (
        execution[(execution["dataset"] == dataset) & (execution["query_set"] == "collection")]
        .groupby(["approach", "execution"])
        .agg({"query_collection_time": ["mean", "std"]})
        .values
    )

    ax.bar(
        x=0,
        height=data[3][0],
        width=0.5,
        # yerr=data[3][1],
        color=sns.color_palette()[0],
        edgecolor="black",
        label=r"\pscan{}",
    )
    ax.bar(
        x=0.75,
        height=data[0][0],
        width=0.5,
        # yerr=data[0][1],
        color=sns.color_palette()[1],
        edgecolor="black",
        label=r"\binsort{}",
    )
    ax.bar(
        x=1.5,
        height=data[4][0],
        width=0.5,
        # yerr=data[4][1],
        color=sns.color_palette()[2],
        edgecolor="black",
        hatch="////",
        label=r"\system{} w/ results",
    )
    ax.bar(
        x=2.05,
        height=data[5][0],
        width=0.5,
        # yerr=data[5][1],
        color=sns.color_palette()[2],
        edgecolor="black",
        hatch="oooo",
        label=r"\system{} w/o results",
    )

    ax.set_xticks([])
    ax.set_yscale("log")
    if dataset == "gittables":
        ax.set_ylim(top=ax.get_ylim()[1] * 4)
    else:
        ax.set_ylim(top=ax.get_ylim()[1] * 2.5)
    autolabel_bars(ax, precision=3, decimal_precision=2)

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/runtime_benchmark/query_collection_{dataset}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )

    ax.set_ylabel("Time (s)")
    bbox = fig.get_tightbbox()
    label_bbox = Bbox(((bbox.x0, bbox.y0), (0.07, bbox.y1)))
    plt.savefig("plots/runtime_benchmark/query_collection_ylabel.pdf", bbox_inches=label_bbox)
    plt.close()

handles, labels = ax.get_legend_handles_labels()  # type: ignore
handles.append(copy.deepcopy(handles[-1]))
handles[2].patches[0].set_hatch("")  # type: ignore
handles[3].patches[0].set_facecolor("white")  # type: ignore
handles[3].patches[0].set_hatch("////")  # type: ignore
handles[4].patches[0].set_facecolor("white")  # type: ignore
labels = labels[:2] + [r"\system{}", "w/ results", "w/o results"]

plot_legend(
    "plots/runtime_benchmark/query_collection_legend.pdf", handles=handles, labels=labels, ncol=5
)

### Low Selectivity Comparison

In [9]:
height = 0.8
for dataset in ["sportstables", "open_data_usa", "gittables"]:
    fig, ax = plt.subplots(1, 1, figsize=(1.2, height))
    data = (
        low_selectivity[
            (low_selectivity["dataset"] == dataset)
            & (low_selectivity["query_set"] == "collection")
        ]
        .groupby(["approach", "execution"])
        .agg({"query_collection_time": ["mean", "std"]})
        .values
    )

    ax.bar(
        x=0,
        height=data[3][0],
        width=0.5,
        # yerr=data[3][1],
        color=sns.color_palette()[0],
        edgecolor="black",
        label=r"\pscan{}",
    )
    ax.bar(
        x=0.75,
        height=data[0][0],
        width=0.5,
        # yerr=data[0][1],
        color=sns.color_palette()[1],
        edgecolor="black",
        label=r"\binsort{}",
    )
    ax.bar(
        x=1.5,
        height=data[4][0],
        width=0.5,
        # yerr=data[4][1],
        color=sns.color_palette()[2],
        edgecolor="black",
        hatch="////",
        label=r"\system{} w/ results",
    )
    ax.bar(
        x=2.05,
        height=data[5][0],
        width=0.5,
        # yerr=data[5][1],
        color=sns.color_palette()[2],
        edgecolor="black",
        hatch="oooo",
        label=r"\system{} w/o results",
    )

    # ax.set_xticks([0, 0.75, 1.775], [r"\pscan{}", r"\binsort{}", r"\system{}"])
    ax.set_xticks([])
    ax.set_yscale("log")
    if dataset == "gittables":
        ax.set_ylim(top=ax.get_ylim()[1] * 4)
    else:
        ax.set_ylim(top=ax.get_ylim()[1] * 2.5)
    autolabel_bars(ax, precision=3, decimal_precision=2)

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/runtime_benchmark/low_selectivity_{dataset}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )

    ax.set_ylabel("Time (s)")
    bbox = fig.get_tightbbox()
    label_bbox = Bbox(((bbox.x0, bbox.y0), (0.07, bbox.y1)))
    plt.savefig("plots/runtime_benchmark/low_selectivity_ylabel.pdf", bbox_inches=label_bbox)
    plt.close()

handles, labels = ax.get_legend_handles_labels()  # type: ignore
handles.append(copy.deepcopy(handles[-1]))
handles[2].patches[0].set_hatch("")  # type: ignore
handles[3].patches[0].set_facecolor("white")  # type: ignore
handles[3].patches[0].set_hatch("////")  # type: ignore
handles[4].patches[0].set_facecolor("white")  # type: ignore
labels = labels[:2] + [r"\system{}", "w/ results", "w/o results"]

plot_legend(
    "plots/runtime_benchmark/low_selectivity_legend.pdf", handles=handles, labels=labels, ncol=5
)

### Query Runtime Breakdown

In [10]:
index_trace.groupby(["dataset", "metric"]).agg({"value": ["mean", "std"]})

value          
                                             mean       std
dataset       metric                                       
gittables     execution_time             0.390522  0.005917
              query_bin_search_time      0.000021  0.000016
              query_boostrap_time        0.000005  0.000001
              query_cluster_skip_time    0.003754  0.010311
              query_hist_search_time     0.000078  0.000040
              query_result_update_time   0.002609  0.003235
              query_time                 0.390405  0.005918
              total_time                17.583670  3.628835
open_data_usa execution_time             0.033587  0.005685
              query_bin_search_time      0.000018  0.000010
              query_boostrap_time        0.000004  0.000001
              query_cluster_skip_time    0.000039  0.000142
              query_hist_search_time     0.000011  0.000003
              query_result_update_time   0.000044  0.000107
              query_time                 0.033431  0.005659
              total_time                 0.437602  0.070476
sportstables  execution_time             0.016011  0.001760
              query_bin_search_time      0.000021  0.000013
              query_boostrap_time        0.000004  0.000001
              query_cluster_skip_time    0.000036  0.000058
              query_hist_search_time     0.000012  0.000004
              query_result_update_time   0.000020  0.000009
              query_time                 0.015846  0.001759
              total_time                 0.227551  0.026146

In [11]:
height = 1.2
fig, ax = plt.subplots(1, 1, figsize=(1.79, height))

for i, dataset in enumerate(["sportstables", "open_data_usa", "gittables"]):
    data = (
        index_trace[index_trace.dataset == dataset]
        .groupby(["metric"])
        .agg({"value": ["mean", "std"]})
        .values
    )
    handles = []

    bottom = 0
    for j, idx in enumerate([2, 1, 4, 5, 3]):
        handles += ax.bar(
            x=i * 0.75,
            height=data[idx][0],
            width=0.5,
            bottom=bottom,
            color=sns.color_palette()[j],
            edgecolor="black",
            label=(
                ["Bootstrap", "Bin search", "Histogram search", "Result update", "Cluster skip"][j]
                if i == 0
                else ""
            ),
        )
        bottom += data[idx][0]

    # NOTE: We disregard overhead because it is caused by logging and not the actual computation
    # handles += ax.bar(
    #     x=i * 0.75,
    #     height=data[6][0] - bottom,
    #     width=0.5,
    #     bottom=bottom,
    #     color=sns.color_palette()[5],
    #     edgecolor="black",
    #     label="Overhead" if i == 0 else "",
    # )

    ax.annotate(
        f"{bottom:.4f}",
        xy=(i * 0.75, bottom),  # type: ignore
        xytext=(0, 1),  # 1 point vertical offset
        textcoords="offset points",
        fontsize=6 * 0.8,
        ha="center",
        va="bottom",
    )

ax.set_xticks([0, 0.75, 1.5], ["ST", "OD", "GT"])
# ax.set_ylabel("Time (s)")
ax.set_ylim(
    (index_trace.groupby(["dataset", "metric"]).mean(numeric_only=True).min()).item() / 2,
    ax.get_ylim()[1] * 5,
)
ax.set_yscale("log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc="upper left", fontsize="x-small")
sns.despine()
bbox = fig.get_tightbbox()

plt.tight_layout(pad=1.02)
plt.savefig("plots/runtime_benchmark/runtime_breakdown.pdf", bbox_inches="tight", pad_inches=0.01)
plt.close()

plot_ylabel(
    "plots/runtime_benchmark/runtime_breakdown_ylabel.pdf", "Time (s)", bbox, height, width=0.17
)

### Index Construction Time

In [12]:
pd.set_option("display.max_rows", 20)
(
    indexing[(indexing["parameter"] == "k")]
    .groupby(["dataset", "phase", "parameter_value"])
    .agg({"total_time": ["mean", "std"]})
)

total_time          
                                               mean       std
dataset      phase      parameter_value                      
gittables    clustering 50               102.437872  2.187090
                        60               101.176613  0.946150
                        70               102.382302  1.965855
                        80               101.825904  2.216701
                        90               103.552862  2.861398
...                                             ...       ...
sportstables rebinning  160                7.542296  0.224309
                        170                7.325124  0.163796
                        180                7.526771  0.296109
                        190                7.113499  0.212321
                        200                7.219724  0.466181

[246 rows x 2 columns]

In [13]:
height = 1.1
for i, dataset in enumerate(["sportstables", "open_data_usa", "gittables"]):
    for param, xlabel, x in [
        (
            "k",
            "Number of clusters",
            (
                [1, 2, 5, *list(range(10, 201, 10))],
                [1, 2, 5, *list(range(10, 201, 10))],
                list(range(50, 250, 10)) + list(range(250, 1001, 50)),
            )[i],
        ),
        (
            "b",
            "Bin budget",
            (
                [1000, 5000, 10000, 50000, 100000, 500000, 1000000],
                [1000, 5000, 10000, 50000, 100000, 500000, 1000000],
                [10000, 50000, 100000, 500000, 1000000],
            )[i],
        ),
    ]:
        fig, ax = plt.subplots(1, 1, figsize=(1.77, height))
        for i, (phase, label) in enumerate(
            [
                ("clustering", "Clustering"),
                ("rebinning", "Rebinning"),
                ("conversion", "Conversion"),
            ]
        ):
            data = (
                indexing[
                    (indexing["dataset"] == dataset)
                    & (indexing["parameter"] == param)
                    & (indexing["phase"] == phase)
                ]
                .groupby(["parameter_value"])
                .agg({"total_time": ["mean", "std"]})
                .values
            )

            ax.plot(
                x,
                data[: len(x), 0],
                color=sns.color_palette()[i],
                label=label,
            )
            ax.fill_between(
                x,
                data[: len(x), 0] - data[: len(x), 1],
                data[: len(x), 0] + data[: len(x), 1],
                alpha=0.1,
                edgecolor="white",
                color=sns.color_palette()[i],
            )

        if param == "b":
            ax.set_xscale("log")
            ax.set_yscale("log")
        if param == "k":
            ax.set_xticks([50, 500, 1000])
        ax.set_xlabel(xlabel)
        ax.set_xlim(min(x), max(x))
        handles, labels = ax.get_legend_handles_labels()

        sns.despine()
        plt.tight_layout(pad=1.02)
        bbox = fig.get_tightbbox()
        plt.savefig(
            f"plots/runtime_benchmark/construction_{dataset}_{param}.pdf",
            bbox_inches="tight",
            pad_inches=0.01,
        )
        plt.close()

plot_legend(
    "plots/runtime_benchmark/construction_legend.pdf",
    handles=handles,  # type: ignore
    labels=labels,  # type: ignore
    ncol=5,
)
plot_ylabel(
    "plots/runtime_benchmark/construction_ylabel.pdf",
    "Time (s)",
    bbox,
    height,
    width=0.12,
    xlabel="Number of cluster",
)  # type: ignore

**NOTE:** Bin alignment is incremental, clustering is not.